# 필수과제

- 필수과제 
- 타이타닉 데이터 전처리를 통해서 f1 스코어를 높여주세요!
- 가장 높은 f1 스코어를 만드는 것이 목표
- 모든 분들에게 f1 스코어를 물어보고 공유도 해야 한다.
- 가장 높은 f1 스코어가 나오신 경우는 간단한게 질문할 수 있음
- 교차검증 사용하셔서 구체적인 값까지 나오게해서 정리 꼭 필수입니다.

- 교차검증 횟수는 5번 진행해 주시고
- 모델링은 자유롭게 해주세요! 분류에 필요한 모델링 아무거나 여러가지로 자유롭게 진행해주세요!
- 가장 높은 f1스코어를 찾습니다!

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score #Confusion matrix 수업 때 진행할 예정 
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

In [53]:
import seaborn as sns
df = sns.load_dataset('titanic') #타이타닉 데이터 불러오기

In [54]:
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


###  age, embarked, adult_male, deck, embark_town 결측치 존재

In [5]:
df.columns #컬럼 확인

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [55]:
#train 데이터
train = df[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone']]

In [56]:
#test 데이터
test = df[['survived']]

### 나이 결측치 처리

In [57]:
train['age'].fillna(train['age'].mean(), inplace=True) #평균으로 대체

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\3961318292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['age'].fillna(train['age'].mean(), inplace=True) #평균으로 대체


### 승선항 결측치 처리

In [58]:
train['embarked'].value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [59]:
train['embarked'].fillna('S', inplace=True) #최빈값으로 대체

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\3543687968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['embarked'].fillna('S', inplace=True) #최빈값으로 대체


### adult_male 결측치 처리

In [60]:
train['adult_male'].value_counts()

True     537
False    354
Name: adult_male, dtype: int64

In [61]:
train['adult_male'].fillna(True, inplace=True) #최빈값으로 대체

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\1066649935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['adult_male'].fillna(True, inplace=True) #최빈값으로 대체


## embark_town 결측치 처리

In [62]:
train['embark_town'].value_counts()

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64

In [63]:
train['embark_town'].fillna('Southampton', inplace=True) #최빈값으로 대체

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\3385900538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['embark_town'].fillna('Southampton', inplace=True) #최빈값으로 대체


## deck 결측치 처리

In [64]:
train.drop(['deck'], axis=1, inplace=True) #결측치 너무 많아서 컬럼 삭제

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\2993136790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['deck'], axis=1, inplace=True) #결측치 너무 많아서 컬럼 삭제


## 중복 컬럼들 제거

In [65]:
(train['who'] == train['adult_male']).value_counts() #두개의 컬럼이 동일

False    891
dtype: int64

In [66]:
train.drop(['adult_male'], axis=1, inplace=True) #컬럼 하나 삭제

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\388963390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['adult_male'], axis=1, inplace=True) #컬럼 하나 삭제


In [67]:
(train['sex'] == train['who']).value_counts() #두개의 컬럼이 동일

False    891
dtype: int64

In [68]:
train.drop(['who'], axis=1, inplace=True) #컬럼 하나 삭제

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\1968072463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['who'], axis=1, inplace=True) #컬럼 하나 삭제


In [69]:
#embarked 컬럼과 embark_town 컬럼이 내용상 동일함
train.drop(['embark_town'], axis=1, inplace=True) #컬럼 삭제

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\3636089365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['embark_town'], axis=1, inplace=True) #컬럼 삭제


In [70]:
#survived 컬럼과 alive 컬럼이 내용상 동일함
train.drop(['alive'], axis=1, inplace=True) #컬럼 삭제

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\2892149570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['alive'], axis=1, inplace=True) #컬럼 삭제


In [71]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   pclass    891 non-null    int64   
 1   sex       891 non-null    object  
 2   age       891 non-null    float64 
 3   sibsp     891 non-null    int64   
 4   parch     891 non-null    int64   
 5   fare      891 non-null    float64 
 6   embarked  891 non-null    object  
 7   class     891 non-null    category
 8   alone     891 non-null    bool    
dtypes: bool(1), category(1), float64(2), int64(3), object(2)
memory usage: 50.7+ KB


## 파생변수 생성

In [72]:
train['member'] = train['sibsp']+train['parch'] #가족수 열 추가
train.drop(['sibsp','parch'], axis=1, inplace=True) #컬럼 삭제

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\2289949241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['member'] = train['sibsp']+train['parch'] #가족수 열 추가
C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\2289949241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['sibsp','parch'], axis=1, inplace=True) #컬럼 삭제


In [73]:
train

,pclass,sex,age,fare,embarked,class,alone,member
0,3,male,22.000000,7.2500,S,Third,False,1
1,1,female,38.000000,71.2833,C,First,False,1
2,3,female,26.000000,7.9250,S,Third,True,0
3,1,female,35.000000,53.1000,S,First,False,1
4,3,male,35.000000,8.0500,S,Third,True,0
...,...,...,...,...,...,...,...,...
886,2,male,27.000000,13.0000,S,Second,True,0
887,1,female,19.000000,30.0000,S,First,True,0
888,3,female,29.699118,23.4500,S,Third,False,3
889,1,male,26.000000,30.0000,C,First,True,0


## Scaler

In [74]:
#스케일러
from sklearn.preprocessing import StandardScaler

In [75]:
sds = StandardScaler()

In [76]:
#수치형 변수
numeric_columns = ['pclass','age','member','fare']

In [77]:
df_tit_norm = sds.fit_transform(train[numeric_columns])

In [78]:
df_tit_norm

array([[ 0.82737724, -0.5924806 ,  0.05915988, -0.50244517],
       [-1.56610693,  0.63878901,  0.05915988,  0.78684529],
       [ 0.82737724, -0.2846632 , -0.56097483, -0.48885426],
       ...,
       [ 0.82737724,  0.        ,  1.29942929, -0.17626324],
       [-1.56610693, -0.2846632 , -0.56097483, -0.04438104],
       [ 0.82737724,  0.17706291, -0.56097483, -0.49237783]])

In [97]:
train[numeric_columns] = df_tit_norm

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\4131314809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[numeric_columns] = df_tit_norm


In [98]:
train

,pclass,sex,age,fare,embarked,class,alone,member,EM_C,EM_Q,EM_S
0,0.827377,male,-0.592481,-0.502445,S,Third,False,0.059160,0.0,0.0,1.0
1,-1.566107,female,0.638789,0.786845,C,First,False,0.059160,1.0,0.0,0.0
2,0.827377,female,-0.284663,-0.488854,S,Third,True,-0.560975,0.0,0.0,1.0
3,-1.566107,female,0.407926,0.420730,S,First,False,0.059160,0.0,0.0,1.0
4,0.827377,male,0.407926,-0.486337,S,Third,True,-0.560975,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
886,-0.369365,male,-0.207709,-0.386671,S,Second,True,-0.560975,0.0,0.0,1.0
887,-1.566107,female,-0.823344,-0.044381,S,First,True,-0.560975,0.0,0.0,1.0
888,0.827377,female,0.000000,-0.176263,S,Third,False,1.299429,0.0,0.0,1.0
889,-1.566107,male,-0.284663,-0.044381,C,First,True,-0.560975,1.0,0.0,0.0


## 범주형 변수 처리

In [79]:
#범주형 변수 처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

## 성별

In [81]:
#성별
train['sex'].unique()

array(['male', 'female'], dtype=object)

In [82]:
enc = LabelEncoder()

In [99]:
enc.fit_transform(train['sex'])

array([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,

In [100]:
train['sex'] = enc.fit_transform(train['sex'])

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\1102915103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['sex'] = enc.fit_transform(train['sex'])


## 승선항

In [85]:
#승선항
train['embarked'].unique()

array(['S', 'C', 'Q'], dtype=object)

In [86]:
onehot_encoder = OneHotEncoder()

In [87]:
encoded_em = onehot_encoder.fit_transform(train['embarked'].values.reshape(-1,1)) 

In [88]:
onehot_encoder.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [89]:
onehot_encoder.categories_[0]

array(['C', 'Q', 'S'], dtype=object)

In [102]:
# EM_S, EM_C, EM_Q로 칼럼을 만든다.
em_columns = ['EM_' + em for em in onehot_encoder.categories_[0]]

In [103]:
train[em_columns] = encoded_em.toarray()

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\1387003079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[em_columns] = encoded_em.toarray()


In [105]:
train.drop(['embarked'], axis=1, inplace=True)

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\677336299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['embarked'], axis=1, inplace=True)


In [106]:
train

,pclass,sex,age,fare,class,alone,member,EM_C,EM_Q,EM_S
0,0.827377,1,-0.592481,-0.502445,Third,False,0.059160,0.0,0.0,1.0
1,-1.566107,0,0.638789,0.786845,First,False,0.059160,1.0,0.0,0.0
2,0.827377,0,-0.284663,-0.488854,Third,True,-0.560975,0.0,0.0,1.0
3,-1.566107,0,0.407926,0.420730,First,False,0.059160,0.0,0.0,1.0
4,0.827377,1,0.407926,-0.486337,Third,True,-0.560975,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,-0.369365,1,-0.207709,-0.386671,Second,True,-0.560975,0.0,0.0,1.0
887,-1.566107,0,-0.823344,-0.044381,First,True,-0.560975,0.0,0.0,1.0
888,0.827377,0,0.000000,-0.176263,Third,False,1.299429,0.0,0.0,1.0
889,-1.566107,1,-0.284663,-0.044381,First,True,-0.560975,1.0,0.0,0.0


## 중복 컬럼 처리

In [115]:
#placss 컬럼과 class 컬럼이 내용상 동일함
train.drop(['class'], axis=1, inplace=True) #컬럼 삭제

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\2876939344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['class'], axis=1, inplace=True) #컬럼 삭제


In [116]:
train

,pclass,sex,age,fare,alone,member,EM_C,EM_Q,EM_S
0,0.827377,1,-0.592481,-0.502445,False,0.059160,0.0,0.0,1.0
1,1.000000,1,1.000000,1.000000,False,0.059160,1.0,0.0,0.0
2,0.827377,0,-0.284663,-0.488854,True,-0.560975,0.0,0.0,1.0
3,1.000000,1,1.000000,1.000000,False,0.059160,0.0,0.0,1.0
4,0.827377,1,0.407926,-0.486337,True,-0.560975,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
886,-0.369365,1,-0.207709,-0.386671,True,-0.560975,0.0,0.0,1.0
887,1.000000,1,1.000000,1.000000,True,-0.560975,0.0,0.0,1.0
888,0.827377,0,0.000000,-0.176263,False,1.299429,0.0,0.0,1.0
889,1.000000,1,1.000000,1.000000,True,-0.560975,1.0,0.0,0.0


In [117]:
#alone 컬럼은 member 컬럼에 내용이 포함될 것 같아서 삭제
train.drop(['alone'], axis=1, inplace=True) #컬럼 삭제

C:\Users\dlagm\AppData\Local\Temp\ipykernel_19836\3847711902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['alone'], axis=1, inplace=True) #컬럼 삭제


## 완성된 데이터

In [118]:
train

,pclass,sex,age,fare,member,EM_C,EM_Q,EM_S
0,0.827377,1,-0.592481,-0.502445,0.059160,0.0,0.0,1.0
1,1.000000,1,1.000000,1.000000,0.059160,1.0,0.0,0.0
2,0.827377,0,-0.284663,-0.488854,-0.560975,0.0,0.0,1.0
3,1.000000,1,1.000000,1.000000,0.059160,0.0,0.0,1.0
4,0.827377,1,0.407926,-0.486337,-0.560975,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
886,-0.369365,1,-0.207709,-0.386671,-0.560975,0.0,0.0,1.0
887,1.000000,1,1.000000,1.000000,-0.560975,0.0,0.0,1.0
888,0.827377,0,0.000000,-0.176263,1.299429,0.0,0.0,1.0
889,1.000000,1,1.000000,1.000000,-0.560975,1.0,0.0,0.0


In [126]:
features = train.values #독립변수

In [127]:
label = test.values #종속변수

## 교차검증 - StratifiedKfold

In [128]:
from sklearn.model_selection import StratifiedKFold

In [129]:
skf_titanic = StratifiedKFold(n_splits=5)
fold_df_dt = DecisionTreeClassifier(random_state=111)

n_iter = 0
for train_idx, test_idx in skf_titanic.split(features, label):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    #학습진행
    fold_df_dt.fit(X_train, y_train)
    #예측값 진행
    fold_pred = fold_df_dt.predict(X_test)
    
    #점수 진행
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, fold_pred),4)
    precision = np.round(precision_score(y_test, fold_pred, average='macro'),4)
    recall = np.round(recall_score(y_test, fold_pred,average='macro'),4)
    print('\n{} 교차검증정확도 : {}, 교차검증 precision {}, 교차검증 recall {}'.format(n_iter, accuracy, precision, recall))

print(np.mean(accuracy),'평균정확도!') 


1 교차검증정확도 : 0.676, 교차검증 precision 0.6632, 교차검증 recall 0.6688

2 교차검증정확도 : 0.7135, 교차검증 precision 0.7086, 교차검증 recall 0.7205

3 교차검증정확도 : 0.7697, 교차검증 precision 0.7563, 교차검증 recall 0.7603

4 교차검증정확도 : 0.7753, 교차검증 precision 0.7629, 교차검증 recall 0.7564

5 교차검증정확도 : 0.7472, 교차검증 precision 0.7343, 교차검증 recall 0.7377
0.7472 평균정확도!


## Cross

## 의사결정나무 : 0.6704

In [139]:
#의사결정나무
scores = cross_val_score(fold_df_dt, features, label, scoring='f1',cv=5)

print(np.round(scores,4))
mylist = []
for i in range(len(np.round(scores,4))):
    mylist.append(np.round(scores,4)[i])
mean = sum(mylist) / len(mylist)
print(mean)

[0.6027 0.6667 0.705  0.697  0.6809]
0.6704600000000001


In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## 랜덤포레스트 : 0.7071

In [133]:
fold_df_rf =  RandomForestClassifier()

In [145]:
#랜덤포레스트
scores = cross_val_score(fold_df_rf, features, label, scoring='f1',cv=5)

print(np.round(scores,4))
mylist = []
for i in range(len(np.round(scores,4))):
    mylist.append(np.round(scores,4)[i])
mean = sum(mylist) / len(mylist)
print(mean)

C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y wa

[0.6622 0.6713 0.7407 0.7383 0.7234]
0.7071799999999999


## 로지스틱회귀 : 0.6850

In [135]:
fold_df_lr = LogisticRegression()

In [146]:
#로지스틱회귀
scores = cross_val_score(fold_df_lr, features, label, scoring='f1',cv=5)

print(np.round(scores,4))
mylist = []
for i in range(len(np.round(scores,4))):
    mylist.append(np.round(scores,4)[i])
mean = sum(mylist) / len(mylist)
print(mean)

[0.6443 0.6571 0.72   0.6939 0.7101]
0.6850799999999999


C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

In [143]:
from sklearn.ensemble import GradientBoostingClassifier

## 그라디언트부스팅 : 0.7066

In [144]:
fold_df_gb = GradientBoostingClassifier(max_depth=1)

In [147]:
#그라디언트부스팅
scores = cross_val_score(fold_df_gb, features, label, scoring='f1',cv=5)

print(np.round(scores,4))
mylist = []
for i in range(len(np.round(scores,4))):
    mylist.append(np.round(scores,4)[i])
mean = sum(mylist) / len(mylist)
print(mean)

C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.6197 0.68   0.732  0.7619 0.7397]
0.70666


C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dlagm\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
